# Segmenting and Clustering Neighborhoods in Toronto

## Part 3

### Explore and cluster the neighborhoods in Toronto

Whereas it is up to us which borough to choose to work with, I decided to explore Scarborough. 

*Scarborough* is a northern est part of Toronto. It is a large, multicultural area that contains the Scarborough Bluff, huge cliffs overlooking Lake Ontario, lined with parks, beaches, and hiking trails. The area is also known for its diverse spread of restaurants, including regional Southeast Asian, Chinese, and Indian cuisine.


In [1]:
# Let's start with importing libraries
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

Let's use the data obtained in Part 1 and Part 2. The result data frame that contains data of Toronto's Boroughs, Neighborhoods, and corresponding coordinate was saved as csv file (Toronto_Data.csv) and uploaded to Coursera_Capstone repository on my GitHub.

In [2]:
toronto_df = pd.read_csv("https://raw.githubusercontent.com/Ok-sa/Coursera_Capstone/main/Toronto_Data.csv")
toronto_df.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
toronto_df.dtypes

Unnamed: 0         int64
PostalCode        object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

In [4]:
#drop the excessive column
toronto_df.drop(columns=['Unnamed: 0'], inplace = True)
toronto_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Let's see what uniques boroughs we have:

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [6]:
#find Toronto's coordinate
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


Let's visualise Toronto and its neighborhoods on a map

In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
#let's slice the original dataframe and create a new dataframe of the Scarborough data
scarborough_data = toronto_df[toronto_df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [9]:
#find Scarborough's coordinate
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


In [10]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

## Scarborough exploring

Let's utilize the Foursquare API to explore the neighborhoods and segment them

In [11]:
#define Foursquare Credentials and Version
CLIENT_ID = 'cliend_id' # your Foursquare ID
CLIENT_SECRET = 'client_secret' # your Foursquare Secret
VERSION = '20210116' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2JXW31YXC1QW0UQJ23GS5PMECSGTXEM3EZC1AESQY02XTCMH
CLIENT_SECRET:YYOHDYPO21IWB5MTINH31K1HZHZKXXHU2ABKDPWTAGX3XPMD


Get the neighborhood's name

In [12]:
scarborough_data.loc[0, 'Neighbourhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values

In [13]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.8066863, -79.19435340000003.


Now, let's get the top 100 venues that are in Mavern and Rouge within a radius of 1000 meters

In [14]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2JXW31YXC1QW0UQJ23GS5PMECSGTXEM3EZC1AESQY02XTCMH&client_secret=YYOHDYPO21IWB5MTINH31K1HZHZKXXHU2ABKDPWTAGX3XPMD&v=20210116&ll=43.8066863,-79.19435340000003&radius=1000&limit=100'

Les send the GET request and see what we get)

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600391eb378c523e4a003850'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.815686309000014,
    'lng': -79.18190576146083},
   'sw': {'lat': 43.79768629099999, 'lng': -79.20680103853923}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs'

Clean the json and structure it into a _pandas_ dataframe

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Harvey's,Restaurant,43.800020,-79.198307
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
4,RBC Royal Bank,Bank,43.798782,-79.197090
5,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
6,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
7,Tim Hortons,Coffee Shop,43.802000,-79.198169
8,Lee Valley,Hobby Shop,43.803161,-79.199681
9,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296


In [18]:
nearby_venues.shape

(20, 4)

### Explore Neighborhoods in Scarborough

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
#scarborough_venues
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [21]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [22]:
scarborough_venues.shape

(94, 7)

As we can see, we have 94 venues in within Scarborough borough in Toronto. Let's check how many venues were returned for each neighborhood, adn how many uniques categories

In [26]:
#let's see how many unique categories of venues each neighborhood has
scarborough_venues[['Neighborhood','Venue Category']].groupby('Neighborhood').count()

,Venue Category
Neighborhood,
Agincourt,5
"Birch Cliff, Cliffside West",4
Cedarbrae,8
"Clarks Corners, Tam O'Shanter, Sullivan",12
"Cliffside, Cliffcrest, Scarborough Village West",3
"Dorset Park, Wexford Heights, Scarborough Town Centre",8
"Golden Mile, Clairlea, Oakridge",8
"Guildwood, Morningside, West Hill",8
"Kennedy Park, Ionview, East Birchmount Park",6


In [27]:
#how many uniques categories of venues entire Scarborough has
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 58 uniques categories.


### Analyze Each Neighborhood

In [28]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [29]:
scarborough_onehot.shape

(94, 59)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.0,0.000,0.000000,0.0,0.200000,0.000,0.000,0.000000,0.00,0.000,0.000000,0.2,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.2,0.000,0.2,0.000,0.000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.20,0.0,0.000,0.000000,0.000000,0.000
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000,0.000,0.000000,0.25,0.000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.25,0.0,0.000,0.000000,0.000000,0.000
2,Cedarbrae,0.000000,0.125,0.0,0.125,0.125000,0.0,0.000000,0.000,0.000,0.000000,0.00,0.125,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.125000,0.000000,0.125000,0.00,0.125,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.0,0.000,0.125000,0.000000,0.000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.0,0.000,0.083333,0.0,0.000000,0.000,0.000,0.000000,0.00,0.000,0.083333,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.083333,0.083333,0.000000,0.083333,0.00,0.000,0.000000,0.000,0.000000,0.083333,0.083333,0.0,0.000000,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.000000,0.083333,0.000000,0.000,0.083333,0.166667,0.000000,0.000,0.000,0.000000,0.00,0.0,0.000,0.083333,0.000000,0.000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.0,0.000,0.000000,0.0,0.000000,0.000,0.000,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.333333,0.333333,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.0,0.000,0.000000,0.0,0.000000,0.125,0.000,0.000000,0.00,0.000,0.125000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.125000,0.000000,0.00,0.000,0.000000,0.000,0.250000,0.000000,0.000000,0.0,0.000000,0.0,0.125,0.0,0.000,0.000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.125
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.0,0.250,0.000000,0.0,0.000000,0.000,0.125,0.000000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.125,0.000000,0.125000,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000,0.125,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.125000,0.000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.0,0.125,0.000000,0.

In [31]:
scarborough_grouped.shape

(16, 59)

Let's print each neighborhood along with the top 3 most common venues

In [32]:
num_top_venues = 3

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25


----Cedarbrae----
             venue  freq
0           Bakery  0.12
1             Bank  0.12
2  Thai Restaurant  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
          venue  freq
0   Pizza Place  0.17
1  Intersection  0.08
2   Gas Station  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1                Motel  0.33
2        Movie Theater  0.33


----Dorset Park, Wexford Heights, Scarborough Town Centre----
                   venue  freq
0      Indian Restaurant  0.25
1  Vietnamese Restaurant  0.12
2                Brewery  0.12


----Golden Mile, Clairlea, Oakridge----
          venue  f

Now let's create the new dataframe and display the top 5 venues for each neighborhood

In [33]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
#create the new dataframe and display the top 5 venues for each neighborhood

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant
2,Cedarbrae,Gas Station,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Fast Food Restaurant,Noodle House,Thai Restaurant,Gas Station
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Movie Theater,Ice Cream Shop,Hakka Restaurant


### Cluster Neighborhoods

Run _k_-means to cluster the neighborhood

In [35]:
# set number of clusters k = 5
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10]
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 2, 0, 0, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [36]:
neighborhoods_venues_sorted.dtypes

Neighborhood             object
1st Most Common Venue    object
2nd Most Common Venue    object
3rd Most Common Venue    object
4th Most Common Venue    object
5th Most Common Venue    object
dtype: object

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)


scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarborough_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Hakka Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Bar,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Hakka Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Rental Car Location,Restaurant,Medical Center,Electronics Store,Breakfast Spot
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Hakka Restaurant,General Entertainment
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Gas Station,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery


In [38]:
scarborough_merged.shape

(17, 11)

In [40]:
#drop rows with NaN values
scarborough_merged.dropna(inplace = True)

In [41]:
scarborough_merged.shape

(16, 11)

Let's visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #cluster_color = int(cluster)
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



### Examine Clusters

#### Cluster 1

In [43]:
scarborough_merged.loc[scarborough_merged['Cluster Label'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Scarborough,0.0,Rental Car Location,Restaurant,Medical Center,Electronics Store,Breakfast Spot
4,Scarborough,0.0,Gas Station,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery
6,Scarborough,0.0,Discount Store,Coffee Shop,Hobby Shop,Bus Station,Department Store
7,Scarborough,0.0,Bakery,Ice Cream Shop,Bus Line,Metro Station,Soccer Field
8,Scarborough,0.0,American Restaurant,Motel,Movie Theater,Ice Cream Shop,Hakka Restaurant
9,Scarborough,0.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant
10,Scarborough,0.0,Indian Restaurant,Pet Store,Brewery,Chinese Restaurant,Furniture / Home Store
11,Scarborough,0.0,Middle Eastern Restaurant,Auto Garage,Smoke Shop,Bakery,Sandwich Place
12,Scarborough,0.0,Clothing Store,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge
13,Scarborough,0.0,Pizza Place,Fast Food Restaurant,Noodle House,Thai Restaurant,Gas Station


#### Cluster 2

In [44]:
scarborough_merged.loc[scarborough_merged['Cluster Label'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Scarborough,1.0,Bar,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Hakka Restaurant


#### Cluster 3

In [45]:
scarborough_merged.loc[scarborough_merged['Cluster Label'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Scarborough,2.0,Jewelry Store,Playground,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant


#### Cluster 4

In [46]:
scarborough_merged.loc[scarborough_merged['Cluster Label'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Hakka Restaurant


#### Cluster 5

In [47]:
scarborough_merged.loc[scarborough_merged['Cluster Label'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Scarborough,4.0,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Hakka Restaurant,General Entertainment


### Observations report

As we can see, unlike Manhatten (NY, USA), Scarborough borough (Toronto, CA) has less quanity of venues, evenly disperced on a large territory. Like in the most of cities, coffe shops are the most popular venues. 